<a href="https://colab.research.google.com/github/MJU-BDP/young/blob/sooy/RecSys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os

# 경로에 파일 잘 받아졌는지 확인
for dirname, _, filenames in os.walk('/content/drive/MyDrive/movie data'):
  for filename in filenames:
    print(os.path. join(dirname, filename))

/content/drive/MyDrive/movie data/genome_scores.csv
/content/drive/MyDrive/movie data/movie.csv
/content/drive/MyDrive/movie data/link.csv
/content/drive/MyDrive/movie data/genome_tags.csv
/content/drive/MyDrive/movie data/rating.csv
/content/drive/MyDrive/movie data/tag.csv


In [ ]:
movie=pd.read_csv('/content/drive/MyDrive/movie data/movie.csv')
rating=pd.read_csv('/content/drive/MyDrive/movie data/rating.csv')
link=pd.read_csv('/content/drive/MyDrive/movie data/link.csv')
tag=pd.read_csv('/content/drive/MyDrive/movie data/tag.csv')
genome_tags=pd.read_csv('/content/drive/MyDrive/movie data/genome_tags.csv')
genome_scores=pd.read_csv('/content/drive/MyDrive/movie data/genome_scores.csv')

In [ ]:
movie

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
27273,131254,Kein Bund für's Leben (2007),Comedy
27274,131256,"Feuer, Eis & Dosenbier (2002)",Comedy
27275,131258,The Pirates (2014),Adventure
27276,131260,Rentun Ruusu (2001),(no genres listed)


In [ ]:
# | 를 기준으로 split
movie['genres'] = movie.genres.str.split('|')
print(movie. head())
movieFeatures = movie.copy ()
for index, row in movie.iterrows ():
  for genre in row['genres']:
    # 장르가 있으면 1 없으면 0으로 인코딩
    movieFeatures.at[index, genre] = 1

movieFeatures = movieFeatures.fillna(0)
movieFeatures.head()

In [ ]:
# title, year 분리
movie['year'] = movie.title.str.extract('(\(\d\d\d\d\))', expand=False)
movie['year'] = movie.year.str.extract('(\d\d\d\d)', expand=False)
movie['title'] = movie.title.str.replace('(\(\d\d\d\d\))', '')
movie['title'] = movie['title'].apply(lambda x: x.strip())

<ipython-input-47-99ce61c14db9>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  movie['title'] = movie.title.str.replace('(\(\d\d\d\d\))', '')


In [ ]:
movie

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995
...,...,...,...,...
27273,131254,Kein Bund für's Leben,[Comedy],2007
27274,131256,"Feuer, Eis & Dosenbier",[Comedy],2002
27275,131258,The Pirates,[Adventure],2014
27276,131260,Rentun Ruusu,[(no genres listed)],2001


In [ ]:
small_movie = movieFeatures.iloc[:5][["Adventure","Comedy", "Fantasy", "Romance"]]

from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import cosine_similarity
print(jaccard_score(small_movie.iloc[0],small_movie.iloc[1]))
print(cosine_similarity([small_movie.iloc[0],small_movie.iloc[1]]))

0.6666666666666666
[[1.         0.81649658]
 [0.81649658 1.        ]]


In [ ]:
userInput = [
              {'title': 'Breakfast Club, The', 'rating': 5},
              {'title': 'Toy Story', 'rating': 3.5},
              {'title': 'Jumanji','rating': 2},
              {'title': "Pulp Fiction", 'rating':5},
              {'title': 'Akira', 'rating': 4.5}
             ]
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


In [ ]:
inputId = movie[movie['title'].isin(inputMovies['title'].tolist())]
inputMovies = pd.merge(inputId, inputMovies)
inputMovies = inputMovies.drop('genres', 1)
inputMovies

<ipython-input-51-9cc2142f1b18>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  inputMovies = inputMovies.drop('genres', 1)


,movieId,title,year,rating
0,1,Toy Story,1995,3.5
1,2,Jumanji,1995,2.0
2,296,Pulp Fiction,1994,5.0
3,1274,Akira,1988,4.5
4,1968,"Breakfast Club, The",1985,5.0


In [ ]:
userMovies = movieFeatures[movieFeatures['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies = userMovies.reset_index(drop=True)
userMovies = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1)
userMovies

<ipython-input-52-5daf18b01f35>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  userMovies = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1)
<ipython-input-52-5daf18b01f35>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  userMovies = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1)
<ipython-input-52-5daf18b01f35>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  userMovies = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1)


,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
userProfile = userMovies.transpose().dot(inputMovies['rating'])
userProfile

Adventure             10.0
Animation              8.0
Children               5.5
Comedy                13.5
Fantasy                5.5
Romance                0.0
Drama                 10.0
Action                 4.5
Crime                  5.0
Thriller               5.0
Horror                 0.0
Mystery                0.0
Sci-Fi                 4.5
IMAX                   0.0
Documentary            0.0
War                    0.0
Musical                0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

In [ ]:
movieFeatures = movieFeatures.set_index(movieFeatures['movieId'])
movieFeatures = movieFeatures.drop('movieId', 1).drop('title', 1).drop('genres', 1)
movieFeatures.head()

<ipython-input-54-b010025b5cbb>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  movieFeatures = movieFeatures.drop('movieId', 1).drop('title', 1).drop('genres', 1)
<ipython-input-54-b010025b5cbb>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  movieFeatures = movieFeatures.drop('movieId', 1).drop('title', 1).drop('genres', 1)
<ipython-input-54-b010025b5cbb>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  movieFeatures = movieFeatures.drop('movieId', 1).drop('title', 1).drop('genres', 1)


,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
recommendationTable = ((movieFeatures*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable

movieId
1         0.594406
2         0.293706
3         0.188811
4         0.328671
5         0.188811
            ...   
131254    0.188811
131256    0.188811
131258    0.139860
131260    0.000000
131262    0.216783
Length: 27278, dtype: float64

In [ ]:
recommendationTable = recommendationTable.sort_values(ascending=False)
recommendationTable

movieId
5018      0.748252
26093     0.734266
27344     0.720280
6902      0.678322
117646    0.678322
            ...   
106431    0.000000
1856      0.000000
106421    0.000000
106405    0.000000
68069     0.000000
Length: 27278, dtype: float64

In [ ]:
movie.loc[movie['movieId'].isin(recommendationTable.head(20).keys())]

,movieId,title,genres,year
664,673,Space Jam,"[Adventure, Animation, Children, Comedy, Fanta...",1996
1823,1907,Mulan,"[Adventure, Animation, Children, Comedy, Drama...",1998
2901,2987,Who Framed Roger Rabbit?,"[Adventure, Animation, Children, Comedy, Crime...",1988
4922,5018,Motorama,"[Adventure, Comedy, Crime, Drama, Fantasy, Mys...",1991
6792,6902,Interstate 60,"[Adventure, Comedy, Drama, Fantasy, Mystery, S...",2002
8603,26093,"Wonderful World of the Brothers Grimm, The","[Adventure, Animation, Children, Comedy, Drama...",1962
8780,26340,"Twelve Tasks of Asterix, The (Les douze travau...","[Action, Adventure, Animation, Children, Comed...",1976
9291,27344,Revolutionary Girl Utena: Adolescence of Utena...,"[Action, Adventure, Animation, Comedy, Drama, ...",1999
9819,32031,Robots,"[Adventure, Animation, Children, Comedy, Fanta...",2005
11704,51632,Atlantis: Milo's Return,"[Action, Adventure, Animation, Children, Comed...",2003


In [ ]:
rating

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40
...,...,...,...,...
20000258,138493,68954,4.5,2009-11-13 15:42:00
20000259,138493,69526,4.5,2009-12-03 18:31:48
20000260,138493,69644,3.0,2009-12-07 18:10:57
20000261,138493,70286,5.0,2009-11-13 15:42:24


In [ ]:
print(len(rating['userId'].unique()))
print(len(rating['movieId'].unique()))

138493
26744


In [ ]:
x=rating['userId'].value_counts()>500
print(len(x[x]))
thin_rating=rating[rating['userId'].isin(x[x].index)]
movie_pivot=thin_rating.pivot_table(columns='userId',index='movieId',values='rating')

7441


In [ ]:
movie_pivot

userId,11,24,54,58,91,104,116,134,156,208,...,138270,138301,138307,138325,138382,138397,138406,138411,138437,138474
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.5,4.0,4.0,5.0,4.0,NaN,3.0,4.0,5.0,4.0,...,NaN,2.5,3.5,5.0,3.0,NaN,4.0,5.0,4.0,5.0
2,NaN,NaN,3.0,NaN,3.5,NaN,2.0,NaN,5.0,NaN,...,NaN,2.5,2.5,3.0,4.0,NaN,3.0,NaN,NaN,4.0
3,NaN,NaN,NaN,NaN,3.0,NaN,2.0,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,3.0,5.0,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,1.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
5,NaN,2.0,3.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131254,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131256,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
y=thin_rating['movieId'].value_counts()>50
print(len(y[y]))
thin_rating=thin_rating[thin_rating['movieId'].isin(y[y].index)]
movie_pivot=thin_rating.pivot_table(columns='userId',index='movieId',values='rating')

9313


In [ ]:
movie_pivot.fillna(0,inplace=True)
movie_pivot

NameError: ignored

In [ ]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

movie_sparse=csr_matrix(movie_pivot)
model=NearestNeighbors(n_neighbors=10,algorithm='brute',metric='cosine')
model.fit(movie_sparse)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=10)

In [ ]:
movie_pivot.iloc[1,:].values.reshape(1, -1)

array([[0., 0., 3., ..., 0., 0., 4.]])

In [ ]:
distances,suggestions=model.kneighbors(movie_pivot.iloc[1,:].values.reshape(1,-1))
distances,suggestions

(array([[0.        , 0.22626448, 0.23797185, 0.23983944, 0.24134662,
         0.24469272, 0.2489989 , 0.25214387, 0.2534928 , 0.25541199]]),
 array([[   1,  438,  658, 1273,  326,  334,  337, 2112, 1037,  532]]))

In [ ]:
movie[movie['movieId'].isin(movie_pivot.index[suggestions[0]])]

,movieId,title,genres,year
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
352,356,Forrest Gump,"[Comedy, Drama, Romance, War]",1994
360,364,"Lion King, The","[Adventure, Animation, Children, Drama, Musica...",1994
363,367,"Mask, The","[Action, Comedy, Crime, Fantasy]",1994
476,480,Jurassic Park,"[Action, Adventure, Sci-Fi, Thriller]",1993
580,586,Home Alone,"[Children, Comedy]",1990
767,780,Independence Day (a.k.a. ID4),"[Action, Adventure, Sci-Fi, Thriller]",1996
1242,1270,Back to the Future,"[Adventure, Comedy, Sci-Fi]",1985
1528,1580,Men in Black (a.k.a. MIB),"[Action, Comedy, Sci-Fi]",1997
2486,2571,"Matrix, The","[Action, Sci-Fi, Thriller]",1999


In [ ]:
movie_pivot.iloc[suggestions[0]]

userId,11,24,54,58,91,104,116,134,156,208,...,138270,138301,138307,138325,138382,138397,138406,138411,138437,138474
movieId,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,3.0,0.0,3.5,0.0,2.0,0.0,5.0,0.0,...,0.0,2.5,2.5,3.0,4.0,0.0,3.0,0.0,0.0,4.0
480,5.0,3.0,5.0,4.0,3.5,0.0,4.0,4.0,5.0,4.5,...,4.0,3.0,3.5,5.0,5.0,4.5,4.0,4.0,0.0,5.0
780,5.0,4.0,5.0,2.0,3.0,0.0,1.0,3.0,5.0,4.0,...,3.0,1.5,3.0,3.0,5.0,5.0,3.0,2.5,4.5,0.0
1580,5.0,5.0,4.0,4.0,3.5,3.0,2.5,4.0,5.0,1.5,...,3.0,2.0,3.0,4.5,5.0,5.0,3.5,4.0,4.5,5.0
356,5.0,5.0,5.0,4.5,4.0,0.0,4.0,3.0,5.0,1.0,...,4.0,3.5,0.0,4.5,5.0,4.5,4.0,4.0,0.0,5.0
364,4.0,0.0,0.0,0.0,4.5,0.0,4.0,0.0,4.0,3.5,...,4.0,3.0,0.0,5.0,3.0,4.0,4.0,3.5,0.0,5.0
367,4.0,4.0,4.0,5.0,2.5,0.0,1.0,3.0,4.0,3.5,...,3.0,3.0,1.5,0.0,1.0,0.0,3.5,3.5,0.0,5.0
2571,5.0,5.0,5.0,5.0,3.5,1.0,4.0,3.0,5.0,3.0,...,3.0,3.5,4.0,4.0,5.0,4.5,4.0,5.0,5.0,5.0
1270,5.0,4.0,5.0,0.0,4.0,3.0,3.5,3.0,4.0,4.0,...,4.0,3.5,3.5,5.0,4.0,4.5,4.0,4.5,4.0,0.0


In [ ]:
movie_pivot.iloc[suggestions[0,1:]].replace(0, np.nan).mean(skipna=True)

userId
11        4.666667
24        4.285714
54        4.500000
58        3.642857
91        3.500000
            ...   
138397    4.571429
138406    3.777778
138411    3.875000
138437    4.500000
138474    4.857143
Length: 7441, dtype: float64

In [ ]:
movie_pivot_est = movie_pivot.copy()

In [ ]:
movie_pivot_est.loc[2] = movie_pivot.iloc[suggestions[0,1:]].replace(0, np.nan).mean(skipna=True)
movie_pivot_est

userId,11,24,54,58,91,104,116,134,156,208,...,138270,138301,138307,138325,138382,138397,138406,138411,138437,138474
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.500000,4.000000,4.0,5.000000,4.0,0.000000,3.000000,4.000000,5.000000,4.000000,...,0.000000,2.500000,3.500000,5.0000,3.0,0.000000,4.000000,5.000,4.0,5.000000
2,4.666667,4.285714,4.5,3.642857,3.5,2.333333,2.777778,3.285714,4.555556,3.166667,...,3.333333,2.888889,3.083333,4.3125,4.0,4.571429,3.777778,3.875,4.5,4.857143
3,0.000000,0.000000,0.0,0.000000,3.0,0.000000,2.000000,0.000000,2.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,3.0,5.000000,0.000000,0.000,0.0,0.000000
4,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,3.000000,0.000000,...,1.000000,0.000000,0.000000,0.0000,4.0,0.000000,0.000000,0.000,0.0,0.000000
5,0.000000,2.000000,3.0,0.000000,0.0,0.000000,0.000000,0.000000,3.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000,0.0,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118700,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000,0.0,0.000000
118900,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000,0.0,0.000000
118997,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000,0.0,0.000000
